# MEMS-TFT

## Introduction

In this notebook we program the MEMS-TFT PCB, which has 1 STHV1600 IC for pulsing and 3 level shifter units for driving the TFT matric

In [1]:
#import of all libraries
import threading
import logging
import asyncio
import numpy as np
from pynq import Overlay
from pynq.overlays.ip import *
from pynq.overlays.ip.STHV1600 import sthv1600_regmap as reg
import pynq.interrupt
import json

we import the bitstream as an overlay

In [5]:
# ol = Overlay('/home/xilinx/pynq/overlays/MEMS_TFT/base.bit')
ol = Overlay('/usr/local/lib/python3.6/dist-packages/carpatchiot_mems_tft/bitstream/mems_tft.bit')
ol?

In [3]:
#sthv is the main point of contact

leds = ol.gpio_leds
switches = ol.gpio_switches
buttons = ol.gpio_buttons
axi_intr = ol.axi_intc_0
BRAM = ol.axi_bram_ctrl_0

sthv = ol.STHV1600_0
prg = ol.STHV1600_0.S00_AXI_PRG
trg = ol.STHV1600_0.S01_AXI_TRIG
intr = ol.STHV1600_0.S_AXI_INTR
# sthv?
sthv.connect_BRAM(BRAM)
TFT= ol.tft_states_0
# TFT?
sthv.connect_TFT(TFT)

connecting TFT


In [4]:
def read_test():
    res = sthv.STHV_RAM_READ(0x1, reg.DEVID0, 0x1003)
    if(res[0] != 0xf7):
        print("error in data reading of STHV1600", hex(res[0]))
    else:
        print("read is OK",hex(res[0]), "@ reg 0x1003")
#     res = sthv.STHV_RAM_READ(amount=1, devId=reg.DEVID0,address=reg.config_1)
    print("read result: ",hex(res[0]))
read_test()

-->reading from reg 0x801c0
error in data reading of STHV1600 0xffff
read result:  0xffff


In [29]:
sthv.initRAM(devId=reg.DEVID0)

In [30]:
pynq.ps.Clocks.fclk0_mhz=150

In [31]:
devIdList = [reg.DEVID0]

# example use case

<code>
def example_case(self, devIdList, topology):
        volt_states = [reg.Level_State.HVP1.value, reg.Level_State.CLAMP.value,
                       reg.Level_State.NOP.value, reg.Level_State.NOP.value]
        time_reg_ns = [250, 200, 0, 0] 
        time_reg_ticks = nanos_to_ticks(time_reg_ns)
        print("time_reg_ticks",time_reg_ticks)
        amount_of_states = 2
        repetitions = 2
        delay_table = [10, 220, 30, 140, 250]
        for i in devIdList:
            self.enableTx(devId=i, onChannels=0xFF)
            self.start_delay(devId=i,delay_table_nsec=delay_table)
            self.programPulseAllChannels(
                i, volt_states, time_reg_ticks, amount_of_states, repetitions)
            result = self.STHV_RAM_READ(amount_of_states, i,reg.channel_0)
            print(hex(i) for i in result)
        if(topology=='MEMS-ONLY'):
            txtime = self.calc_tx_time_ticks(time_reg_ticks)
            self.STHV_TRIG(txtime, 0x4000, 10)
        else:
            if(topology=='MEMS-TFT'):
                txtime = self.calc_tx_time_ticks(time_reg_ticks)
                self.STHV_TRIG(txtime, 0x4000, 10)
                self.TFT.init_joint_passive(0x0, 0x0, 0x200, 0x1000, 5) # 0x200=512: 5.25us with 100MHz clock, 0x1000=4096: 41.1us
            else:
                print("wrong topology name, use 'MEMS-TFT' or 'MEMS-ONLY'")
</code>

In [32]:
sthv.example_case(devIdList=devIdList, topology='MEMS-TFT',cycle_amount=5)

time_reg_ticks [25, 20, 0, 0]
start delay: [1, 22, 3, 14, 25]
amount of pulse regs: 3
--> pulse shape registers per channel:  0x4996
--> pulse shape registers per channel:  0x14f
--> pulse shape registers per channel:  0xe
-->reading from reg 0x80440
read 0x 0x0
MEMS-TFT programming
programming STHV1600 trigger in ticks mode 160
joint passive TFT initiated


In [17]:
# sthv.low_power_test(devId=reg.DEVID0)

In [18]:
sthv.STHV_TRIG(cycle_amount=1,idle_time=0x4000, tx_rxtime=0x200)

programming STHV1600 trigger in ticks mode 512


In [19]:
sthv.STHV_PRG_RAM(devId=reg.DEVID0, address=0x1008, amount=1, data=0x0000)

In [20]:
TFT.init_joint_passive(data_l=0x0, read_sel_l=0x0, time_sel=0x0001, time_restart=0xffff, cycle_amount=100000)

joint passive TFT initiated


In [21]:
def nanos_to_ticks(data):
    clk = pynq.ps.Clocks.fclk0_mhz/reg.clock_offset_error
    if(type(data)==list):
        data= [int(i/(clk/10)) for i in data]
    else:
        data = int(data/int(clk/10))
    return data

In [22]:
pynq.ps.Clocks.fclk0_mhz=150
devIdList = [reg.DEVID0]
def example_case(devIdList, topology):
    volt_states = [reg.Level_State.HVP1.value, reg.Level_State.CLAMP.value,
                           reg.Level_State.HVP1.value, reg.Level_State.CLAMP.value]
    time_reg_ns = [100, 200]#, 20, 10] 
    time_reg_ticks = nanos_to_ticks(time_reg_ns)
    print("time_reg_ticks",time_reg_ticks)
    amount_of_states = 2
    repetitions = 4 # becomes 5 repetitions
    delay_table = [10, 20, 30, 40,50]
    for i in devIdList:
        sthv.enableTx(devId=i, onChannels=0xFF)
        sthv.enableRx(devId=i, onChannels=0xFF)
        sthv.start_delay(devId=i,delay_table_nsec=delay_table)
        sthv.programPulseAllChannels(
            i, volt_states, time_reg_ticks, amount_of_states, repetitions)
        #define start point register
        sthv.STHV_PRG_RAM(devId=i, amount=1,address=reg.pointer_channel_1_0,data=0x0000)
        result = sthv.STHV_RAM_READ(amount_of_states, i,reg.channel_0)
        print(hex(i) for i in result)
    txtime = sthv.calc_tx_time_ticks([time_reg_ticks], repetitions+2, delay_table)
    sthv.STHV_TRIG(txtime, 0x1000, 10)
#     sthv.STHV_TRIG(0x5000, 0x1000, 10)

#     if(topology=='MEMS-ONLY'):
#         txtime = sthv.calc_tx_time_ticks([time_reg_ticks], delay_table)
#         sthv.STHV_TRIG(txtime, 0x1000, 10)
#     else:
    if(topology=='MEMS-TFT'):

        TFT.init_joint_passive(data_l=0xC, read_sel_l=0xC, time_sel=0x200, time_restart=0x100, cycle_amount=3)
#     else:
#         print("wrong topology name, use 'MEMS-TFT' or 'MEMS-ONLY'")
example_case(devIdList=devIdList, topology='MEMS-TFT')

time_reg_ticks [10, 20]
start delay: [1, 2, 3, 4, 5]


AttributeError: Could not find IP or hierarchy programPulseAllChannels in overlay

In [23]:
sthv.example_datasheet_21(devId=reg.DEVID0,topology='MEMS-TFT')
TFT.init_joint_passive(data_l=0x0, read_sel_l=0x0, time_sel=0x200, time_restart=0x100, cycle_amount=5)

programming STHV1600 trigger in ticks mode 48
joint passive TFT initiated
joint passive TFT initiated


### test tft lines
a function fixing the data lines in a certain state.

A binary '1' means that the line is activated (V+ at the NMOS, so open), a binary '0' means that the line is disactivated (V- at the NMOS so closed)

In [24]:
TFT.test_tft_lines(data_l=0xf,drive_sel_l=0x0,read_sel_l=0x0)

testing TFT initiated


In [16]:
TFT.read(0x1c)

1